In [52]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


In [24]:
templete = PromptTemplate.from_template(
  "{city1}과 {city2}의 거리는 얼마나 되나요?"
)

prompt = templete.format(city1="서울", city2="도쿄")

chat.predict(prompt)

templete = ChatPromptTemplate.from_messages([
  ("system", "당신은 지리전문가입니다. 그리고 당신은 {language}로만 사용해서 대답합니다. "), # 시스템 메시지
  ("ai", "안녕하세요. 저는 지리전문가 {name}입니다. 무엇을 도와드릴까요?"),             # 초기 메모리
  ("human", "{city1}과 {city2}의 거리는 얼마나 되나요? 그리고 당신의 이름은 무엇인가요?"),   # 사용자 메시지
])


prompt = templete.format_messages(language="한국어", name="지쿠", city1="서울", city2="도쿄")

chat.predict_messages(prompt)

AIMessage(content='서울과 도쿄의 직선 거리는 약 1,200km 정도입니다. 제 이름은 지쿠입니다. 어떤 다른 질문이 있나요?')

In [33]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
  def parse(self, text: str) -> str:
    items = text.strip().split(",")
    return [item.strip() for item in items]


In [50]:
templete = ChatPromptTemplate.from_messages([
  ("system", "당신은 리스트 생성 머신입니다. 당신의 대답하는 문장을 콤마로 구분되는 단어 리스트로 만들어주세요. 최대 {max_items}개 만큼 만들어주세요. 모든 값은 중복되지 않아야합니다."), # 시스템 메시지
  ("human", "{question}"),   # 사용자 메시지
])


templete2 = ChatPromptTemplate.from_messages([
  ("system", "당신은 영어 학습 마스터입니다. 리스트값을 기반으로 갖고 영어단어로 번역해주세요. 그리고 한국어 뜻도 같이 배열로 출력해주세요. 예시: [Seoul: 서울시], [Tokyo: 도쿄시]"), # 시스템 메시지
  ("human", "{listItems}"),   # 사용자 메시지
])


In [55]:
w_chain = templete | chat 
e_chain = templete2 | chat 

f_chain = {"listItems": w_chain} | e_chain

result = f_chain.invoke({"max_items": "10", "question": "golang을 공부하는 방법은 무엇인가요?"})

result.content

1. 온라인,
2. 공식문서,
3. 프로젝트,
4. 커뮤니티,
5. 튜토리얼,
6. 책,
7. 실습,
8. 강의,
9. 스터디그룹,
10. 블로그['online: 온라인', 'official document: 공식문서', 'project: 프로젝트', 'community: 커뮤니티', 'tutorial: 튜토리얼', 'book: 책', 'practice: 실습', 'lecture: 강의', 'study group: 스터디그룹', 'blog: 블로그']

"['online: 온라인', 'official document: 공식문서', 'project: 프로젝트', 'community: 커뮤니티', 'tutorial: 튜토리얼', 'book: 책', 'practice: 실습', 'lecture: 강의', 'study group: 스터디그룹', 'blog: 블로그']"